# Interactive Exploratory Data Analysis Dashboard

## In order for the notebook to run properly without callback errors, please select the column needed firstly

### Introduction
This Jupyter Notebook is designed to create an interactive dashboard for Exploratory Data Analysis (EDA) of a given dataset. The dataset is assumed to be related to the banking sector with features like balance, income, age, etc. Also, the dataset contains a date-time column, which is used to create various time-based aggregates and visualizations.

The dashboard provides a variety of options for the user to choose and customize their EDA. These options include selecting the numeric column to explore, choosing the column to group the data by, selecting aggregation functions for time series analysis, specifying a range of dates to focus on, etc. Furthermore, the dashboard will automatically select the top five most interesting columns for the initial exploration, based on a scoring function that takes into account features like changes over time and uneven distributions.

### Expected Input and Output
A pandas DataFrame df that contains your dataset. This DataFrame is expected to include a column with date-time information.
Output
The output is a set of interactive graphs that provide insights about the data. These graphs include histograms, time-series, monthly averages, group time series, correlation graphs, weekday averages, yearly trends, box plots for different time frames, etc.

### Instructions to use
Follow these steps to use the dashboard:

Import the necessary libraries and load your dataset into a pandas DataFrame named df.
Run the cells to preprocess the data and start the dashboard.
The dashboard will open in a new browser window.
Use the dropdown menus to select the column you want to explore, the column to group the data by, the aggregation functions for time series analysis, and the numeric column for correlation analysis.
Use the range slider to select the date range you want to focus on.
The graphs will update automatically based on your selections.
Note: The dashboard assumes that your DataFrame df already exists in your environment and has been preprocessed as necessary before starting the dashboard. For example, missing values have been handled, categorical variables have been appropriately encoded if necessary, etc. The Time column is expected to be in the datetime format. If it's not, you need to convert it before starting the dashboard.

### Notice:

Note:
Since bach date lacks enough date data, I used the cancellation date column as a base for the date, which can be modified.

Since the dataset is too small, there are not enough numbers, and there are too many clusters of categorical variables, some of the line graphs about clusters do not exist because there are no two points that can be connected into a straight line (all are unique points of a certain cluster), and some have only one segment because there are not three or more valid points that can be connected into two segments.

The age in df is not a number and there is no column for marital status and loan status, so this cannot be completed in this dataset.

Explanation:
To find the five most interesting columns, I first defined what is interesting, which I defined as the absolute value of the standard deviation and skewness of each column, calculated the result, and then sorted it to select the five highest scoring columns.



In [1]:
import pandas as pd
import os
from jupyter_dash import JupyterDash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import dash_bootstrap_components as dbc
import gdown
import zipfile


In [4]:
output_file = "feature_store_full_data.zip"
folder_path = os.path.join(os.getcwd(),'data')
if not os.path.exists(folder_path):
    os.mkdir(folder_path)

data_folder_name = 'featurestore_data'
folder_path = os.path.join(folder_path,data_folder_name)
if not os.path.exists(folder_path):
    os.mkdir(folder_path)

    file_id = "1k2b0EHMDGu-3s6f4kJcQ3e-phaR4LKlt"
    url = f"https://drive.google.com/uc?id={file_id}"
    gdown.download(url, output_file)

    zip_file_path = "./" + output_file  
    
    destination_folder = folder_path 

    # Open the zip file
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        # Extract all the contents of the zip file to the destination folder
        zip_ref.extractall(destination_folder)

    os.remove(output_file)



Downloading...
From (uriginal): https://drive.google.com/uc?id=1k2b0EHMDGu-3s6f4kJcQ3e-phaR4LKlt
From (redirected): https://drive.google.com/uc?id=1k2b0EHMDGu-3s6f4kJcQ3e-phaR4LKlt&confirm=t&uuid=453f0eb6-d1ed-4cf9-821c-1504b4739ebf
To: /Users/eligottlieb/Documents/Insait/unified_eda_repository/feature_store_full_data.zip
100%|██████████| 858M/858M [01:20<00:00, 10.7MB/s]


In [5]:
subfolder_name = 'out'
data_folder_name = os.path.join(destination_folder, subfolder_name)
dfs = []
for file in os.listdir(data_folder_name):
    dfs.append(pd.read_parquet(os.path.join(data_folder_name,file)))
    # print('out/' + file)

In [6]:
for df in dfs:
    print(df.shape)

## This makes sense. Different version of data.
## @MAX: it's different version of mock data mixed up, when data got updated out folder should be cleared from old data

(7812, 548)
(7710, 548)
(7192, 548)
(7268, 548)
(6750, 548)
(8346, 548)
(6416, 548)
(8448, 548)
(6328, 548)
(9032, 548)


In [5]:
df = pd.concat(dfs)
print(df.shape)

(2606036, 548)


In [6]:
for col in df.columns:
    if pd.api.types.is_integer_dtype(df[col]) and df[col].dtype == np.int32:
        df[col] = df[col].astype('int64')


In [7]:
pd.set_option('display.max_columns', 15000)

In [ ]:
na_counts = df.isnull().sum()

for i in range(len(na_counts)):
    print(na_counts.index[i], na_counts[i])


In [9]:
df = df.dropna(axis='columns')

In [ ]:
from IPython.display import display
with pd.option_context('display.min_rows', 200):
    display(df.head())

In [11]:
Time = 'AZ_BATCH_DATE'

df[Time] = pd.to_datetime(df[Time])
df['year'] = df[Time].dt.year
df['quarter'] = df[Time].dt.quarter
df['month'] = df[Time].dt.month
df['week'] = df[Time].dt.isocalendar().week
df['day'] = df[Time].dt.day
df['hour'] = df[Time].dt.hour
df['day_of_week'] = df[Time].dt.dayofweek 
df['date'] = df[Time].dt.date

In [1]:
"""
Exploratory Data Analysis Dashboard for Banking Dataset

This script creates a dashboard using Dash and Plotly to perform exploratory data analysis on a banking dataset.
The dashboard allows users to visualize and analyze the data using various graphs and interactive components.

Inputs:
    - df: pandas DataFrame containing the banking dataset

Outputs:
    - Dash application: An interactive dashboard for exploring the dataset

Constants:
    - Time: Name of the column containing the time information

"""




for col in df.columns:
    # Convert specific integer columns to int64 type
    if pd.api.types.is_integer_dtype(df[col]) and (df[col].dtype.name == 'UInt32' or df[col].dtype == np.int32):
        df[col] = df[col].astype('int64')

# Calculate the score for each column
def calculate_score(column):
    return df[column].std() + abs(df[column].skew())

# Get numeric columns only
numeric_columns = [col for col in df.columns if np.issubdtype(df[col].dtype, np.number)]
object_columns = [col for col in df.columns if df[col].dtype == 'object' and col != Time]

# Calculate the score for each column and sort them
scores = {col: calculate_score(col) for col in numeric_columns}
top_columns = sorted(scores, key=scores.get, reverse=True)[:5]

def get_filtered_options():
    new_options = []
    for col in object_columns:
        if df[col].nunique() <= 15:
            new_options.append({'label': col, 'value': col})
    return new_options

filtered_options = get_filtered_options()
default_value = filtered_options[0]['value'] if filtered_options else None

app = JupyterDash(__name__)

app.layout = dbc.Container([
    html.H1("Exploratory Data Analysis of Banking Dataset", style={'textAlign': 'center'}),
    html.P(''' Rows: 3747606, Columns:724
    ''', style={'fontSize': '18px'}),
    html.P('''
        This Dash app is designed to create an interactive dashboard for Exploratory Data Analysis (EDA) of a given dataset. 
        The dataset is assumed to be related to the banking sector with features like balance, income, age, etc. 
        Also, the dataset contains a date-time column, which is used to create various time-based aggregates and visualizations.
    ''', style={'fontSize': '18px'}),
    html.P('''
        The dashboard provides a variety of options for the user to choose and customize their EDA. 
        These options include selecting the numeric column to explore, choosing the column to group the data by, 
        selecting aggregation functions for time series analysis, specifying a range of dates to focus on, etc. 
        Furthermore, the dashboard will automatically select the top five most interesting columns for the initial exploration, 
        based on a scoring function that takes into account features like changes over time and uneven distributions.
    ''', style={'fontSize': '18px'}),

    html.P("What is the distribution of values in the numerical column across all clients? ", 
       style={'fontSize': '25px', 'fontWeight': 'bold'}),
    html.P("What is the average value of the numerical column for all clients?", 
       style={'fontSize': '25px', 'fontWeight': 'bold'}),
    html.P("What is the maximum value of the numerical column for all clients?", 
       style={'fontSize': '25px', 'fontWeight': 'bold'}),
    html.P("What is the minimum value of the numerical column for all clients?", 
       style={'fontSize': '25px', 'fontWeight': 'bold'}),

    
    dbc.Row([
        dbc.Col([
            html.Label('Select Numeric Column:', style={'fontSize': 20}),
            dcc.Dropdown(
                id='numeric-column-select1',
                options=[{'label': i, 'value': i} for i in top_columns + [col for col in numeric_columns if col not in top_columns]],
                value=top_columns
            ),
        ], width=6),
    ]),

    html.Button('Load/Hide Histogram for Distribution, Min, Max, and Mean', id='load-histogram-button', n_clicks=0),
    dcc.Graph(id='histogram'),

    html.P("How does the value of the numerical column vary over time?", style={'fontSize': 25, 'fontWeight': 'bold'}),
    dbc.Row([
        dbc.Col([
            html.Label('Select Numeric Column:', style={'fontSize': 20}),
            dcc.Dropdown(
                id='numeric-column-select2',
                options=[{'label': i, 'value': i} for i in top_columns + [col for col in numeric_columns if col not in top_columns]],
                value=top_columns
            ),
        ], width=6),
        dbc.Col([
            html.Label('Select Aggregation Functions for Time Series:', style={'fontSize': 20}),
            dcc.Dropdown(
                id='agg-func-select',
                options=[{'label': i, 'value': i} for i in ['mean', 'median', 'std']],
                value='mean',
                multi=True
            ),
        ], width=6),
    ]),
    html.Button('Load/Hide Time Series', id='load-time-series-button', n_clicks=0),
    dcc.Graph(id='time-series'),
    
    
    html.P("How does the value of the numerical column change over time for a specific group of clients?", style={'fontSize': 25, 'fontWeight': 'bold'}),
    dbc.Row([
    dbc.Col([
        html.Label('Select Numeric Column:', style={'fontSize': 20}),
        dcc.Dropdown(
            id='numeric-column-select4',
            options=[{'label': i, 'value': i} for i in top_columns + [col for col in numeric_columns if col not in top_columns]],
            value=top_columns
        ),
    ], width=6),

    dbc.Col([
        html.Label('Select Group Column (Object Type):', style={'fontSize': 20}),
        dcc.Dropdown(
            id='group-column-select1',
            options=filtered_options,
            value=default_value
        ),
    ], width=6)
]),
    html.Button('Load/Hide Group Time Series', id='load-group-time-series-button', n_clicks=0),
    dcc.Graph(id='group-time-series'),
    
    html.P("What is the correlation between the numerical column and another numerical column?", style={'fontSize': 25, 'fontWeight': 'bold'}),
    dbc.Row([
        dbc.Col([
            html.Label('Select Numeric Column:', style={'fontSize': 20}),
            dcc.Dropdown(
                id='numeric-column-select5',
                options=[{'label': i, 'value': i} for i in top_columns + [col for col in numeric_columns if col not in top_columns]],
                value=top_columns
            ),
        ], width=6),
        
        dbc.Col([
            html.Label('Select Second Numeric Column for Correlation:', style={'fontSize': 20}),
            dcc.Dropdown(
                id='numeric-column2-select',
                options=[{'label': i, 'value': i} for i in numeric_columns],
                value=numeric_columns[1] if len(numeric_columns) > 1 else numeric_columns[0]
            ),
        ], width=6)
    ]),
    html.Button('Load/Hide Correlation Graph', id='load-correlation-button', n_clicks=0),
    dcc.Graph(id='correlation-graph'),
    
    html.P("What is the average value of the numerical column for each day of the week?", style={'fontSize': 25, 'fontWeight': 'bold'}),
    dbc.Row([
        dbc.Col([
            html.Label('Select Numeric Column:', style={'fontSize': 20}),
            dcc.Dropdown(
                id='numeric-column-select6',
                options=[{'label': i, 'value': i} for i in top_columns + [col for col in numeric_columns if col not in top_columns]],
                value=top_columns
            ),
        ], width=6),
        dbc.Col([
    html.Label('Select Year:', style={'fontSize': 20}),
    dcc.Dropdown(
        id='global-year-select1',
        options=[{'label': 'All Years', 'value': 'ALL'}] +
                [{'label': str(year), 'value': year} for year in df['year'].unique()],
        value=df['year'].min()
    ),
    dcc.DatePickerRange(
    id='date-picker-range-weekday',
    min_date_allowed=df['date'].min(),
    max_date_allowed=df['date'].max(),
    initial_visible_month=df['date'].min(),
    start_date=df['date'].min(),
    end_date=df['date'].max()
)

], width=6)


    ]),
    html.Button('Load/Hide Weekday Average', id='load-weekday-average-button', n_clicks=0),
    dcc.Graph(id='weekday-average'),
    
    
    html.P("What is the distribution of values in the numerical column for different groups of clients?", style={'fontSize': 25, 'fontWeight': 'bold'}),
    dbc.Row([
        dbc.Col([
            html.Label('Select Numeric Column:', style={'fontSize': 20}),
            dcc.Dropdown(
                id='numeric-column-select8',
                options=[{'label': i, 'value': i} for i in top_columns + [col for col in numeric_columns if col not in top_columns]],
                value=top_columns
            ),
        ], width=6),
        dbc.Col([
        html.Label('Select Group Column (Object Type):', style={'fontSize': 20}),
        dcc.Dropdown(
            id='group-column-select2',
            options=filtered_options,
            value=default_value
        ),
    ], width=6)
    ]),
    html.Button('Load/Hide Grouped Box Plot', id='load-grouped-box-plot-button', n_clicks=0),
    dcc.Graph(id='grouped-box-plot'),
    
    
    html.P("What is the distribution of values in the numerical column for each day of the week?", style={'fontSize': 25, 'fontWeight': 'bold'}),
    dbc.Row([
        dbc.Col([
            html.Label('Select Numeric Column:', style={'fontSize': 20}),
            dcc.Dropdown(
                id='numeric-column-select10',
                options=[{'label': i, 'value': i} for i in top_columns + [col for col in numeric_columns if col not in top_columns]],
                value=top_columns
            ),
        ], width=6),
        dbc.Col([
    html.Label('Select Year:', style={'fontSize': 20}),
    dcc.Dropdown(
        id='global-year-select3',
        options=[{'label': 'All Years', 'value': 'ALL'}] +
                [{'label': str(year), 'value': year} for year in df['year'].unique()],
        value=df['year'].min()
    ),
       dcc.DatePickerRange(
        id='date-picker-range',
        min_date_allowed=df['date'].min(),
        max_date_allowed=df['date'].max(),
        initial_visible_month=df['date'].min(),
        start_date=df['date'].min(),
        end_date=df['date'].max()
    ),
], width=6)


    ]),
    html.Button('Load/Hide Day Week Box Plot', id='load-day-week-box-plot-button', n_clicks=0),
    dcc.Graph(id='day-week-box-plot'),
    
    html.P("What is the average value of the numerical column for each year?", style={'fontSize': 25, 'fontWeight': 'bold'}),
    dbc.Row([
        dbc.Col([
            html.Label('Select Numeric Column:', style={'fontSize': 20}),
            dcc.Dropdown(
                id='numeric-column-select11',
                options=[{'label': i, 'value': i} for i in top_columns + [col for col in numeric_columns if col not in top_columns]],
                value=top_columns
            ),
        ], width=6),
    ]),
    html.Button('Load/Hide Yearly Average Bar', id='load-yearly-average-bar-button', n_clicks=0),
    dcc.Graph(id='yearly-average-bar'),
    
    html.P("What is the distribution of values in the numerical column for each quarter of the year? ", style={'fontSize': 25, 'fontWeight': 'bold'}),
    dbc.Row([
        dbc.Col([
            html.Label('Select Numeric Column:', style={'fontSize': 20}),
            dcc.Dropdown(
                id='numeric-column-select12',
                options=[{'label': i, 'value': i} for i in top_columns + [col for col in numeric_columns if col not in top_columns]],
                value=top_columns
            ),
        ], width=6),
        dbc.Col([
    html.Label('Select Year:', style={'fontSize': 20}),
    dcc.Dropdown(
        id='global-year-select4',
        options=[{'label': 'All Years', 'value': 'ALL'}] +
                [{'label': str(year), 'value': year} for year in df['year'].unique()],
        value=df['year'].min()
    ),
], width=6)


    ]),
    html.Button('Load/Hide Quarterly Box Plot', id='load-quarterly-box-plot-button', n_clicks=0),
    dcc.Graph(id='quarterly-box-plot'),
    
    html.P("What is the trend in the value of the numerical column over the days of the week?", style={'fontSize': 25, 'fontWeight': 'bold'}),
    dbc.Row([
        dbc.Col([
            html.Label('Select Numeric Column:', style={'fontSize': 20}),
            dcc.Dropdown(
                id='numeric-column-select13',
                options=[{'label': i, 'value': i} for i in top_columns + [col for col in numeric_columns if col not in top_columns]],
                value=top_columns
            ),
        ], width=6),
        dbc.Col([
    html.Label('Select Year:', style={'fontSize': 20}),
    dcc.Dropdown(
        id='global-year-select5',
        options=[{'label': 'All Years', 'value': 'ALL'}] +
                [{'label': str(year), 'value': year} for year in df['year'].unique()],
        value=df['year'].min()
    ),
    dcc.DatePickerRange(
        id='my-date-picker-range1',
        min_date_allowed=df['date'].min(),
        max_date_allowed=df['date'].max(),
        initial_visible_month=df['date'].min(),
        start_date=df['date'].min(),
        end_date=df['date'].max()
    ),
], width=6)


    ]),
    html.Button('Load/Hide Day of Week Trend', id='load-day-of-week-trend-button', n_clicks=0),
    dcc.Graph(id='day-of-week-trend'),
    
    html.P("What is the distribution of values in the numerical column for each year? ", style={'fontSize': 25, 'fontWeight': 'bold'}),
    dbc.Row([
        dbc.Col([
            html.Label('Select Numeric Column:', style={'fontSize': 20}),
            dcc.Dropdown(
                id='numeric-column-select14',
                options=[{'label': i, 'value': i} for i in top_columns + [col for col in numeric_columns if col not in top_columns]],
                value=top_columns
            ),
        ], width=6),
    ]),
    html.Button('Load/Hide Yearly Distribution', id='load-yearly-distribution-button', n_clicks=0),
    dcc.Graph(id='yearly-distribution'),
    
    html.P("What is the average value of the numerical column for each month of the year?", style={'fontSize': 25, 'fontWeight': 'bold'}),
    dbc.Row([
        dbc.Col([
            html.Label('Select Numeric Column:', style={'fontSize': 20}),
            dcc.Dropdown(
                id='numeric-column-select15',
                options=[{'label': i, 'value': i} for i in top_columns + [col for col in numeric_columns if col not in top_columns]],
                value=top_columns
            ),
        ], width=6),
        dbc.Col([
    html.Label('Select Year:', style={'fontSize': 20}),
    dcc.Dropdown(
        id='global-year-select6',
        options=[{'label': 'All Years', 'value': 'ALL'}] +
                [{'label': str(year), 'value': year} for year in df['year'].unique()],
        value=df['year'].min()
    ),
], width=6)


    ]),
    html.Button('Load/Hide Monthly Average Bar', id='load-monthly-average-bar-button', n_clicks=0),
    dcc.Graph(id='monthly-average-bar'),
    
    html.P("What is the distribution of values in the numerical column for each week of the year?", style={'fontSize': 25, 'fontWeight': 'bold'}),
    dbc.Row([
        dbc.Col([
            html.Label('Select Numeric Column:', style={'fontSize': 20}),
            dcc.Dropdown(
                id='numeric-column-select16',
                options=[{'label': i, 'value': i} for i in top_columns + [col for col in numeric_columns if col not in top_columns]],
                value=top_columns
            ),
        ], width=6),
        dbc.Col([
    html.Label('Select Year:', style={'fontSize': 20}),
    dcc.Dropdown(
        id='global-year-select7',
        options=[{'label': 'All Years', 'value': 'ALL'}] +
                [{'label': str(year), 'value': year} for year in df['year'].unique()],
        value=df['year'].min()
    ),
], width=6)


    ]),
    html.Button('Load/Hide Weekly Distribution', id='load-weekly-distribution-button', n_clicks=0),
    dcc.Graph(id='weekly-distribution'),
    
    html.P("What is the trend in the value of the numerical column over the days of the month?", style={'fontSize': 25, 'fontWeight': 'bold'}),
    dbc.Row([
        dbc.Col([
            html.Label('Select Numeric Column:', style={'fontSize': 20}),
            dcc.Dropdown(
                id='numeric-column-select17',
                options=[{'label': i, 'value': i} for i in top_columns + [col for col in numeric_columns if col not in top_columns]],
                value=top_columns
            ),
        ], width=6),
        dbc.Col([
    html.Label('Select Year:', style={'fontSize': 20}),
    dcc.Dropdown(
        id='global-year-select8',
        options=[{'label': 'All Years', 'value': 'ALL'}] +
                [{'label': str(year), 'value': year} for year in df['year'].unique()],
        value=df['year'].min()
    ),
    dcc.DatePickerRange(
        id='my-date-picker-range2',
        min_date_allowed=df['date'].min(),
        max_date_allowed=df['date'].max(),
        initial_visible_month=df['date'].min(),
        start_date=df['date'].min(),
        end_date=df['date'].max()
    ),
], width=6)


    ]),
    html.Button('Load/Hide Day of Month Trend', id='load-day-of-month-trend-button', n_clicks=0),
    dcc.Graph(id='day-of-month-trend'),
    
    html.P("What is the distribution of values in the numerical column for each month of the year?", style={'fontSize': 25, 'fontWeight': 'bold'}),
    dbc.Row([
        dbc.Col([
            html.Label('Select Numeric Column:', style={'fontSize': 20}),
            dcc.Dropdown(
                id='numeric-column-select18',
                options=[{'label': i, 'value': i} for i in top_columns + [col for col in numeric_columns if col not in top_columns]],
                value=top_columns
            ),
        ], width=6),
        dbc.Col([
    html.Label('Select Year:', style={'fontSize': 20}),
    dcc.Dropdown(
        id='global-year-select9',
        options=[{'label': 'All Years', 'value': 'ALL'}] +
                [{'label': str(year), 'value': year} for year in df['year'].unique()],
        value=df['year'].min()
    ),
], width=6)


    ]),
    html.Button('Load/Hide Monthly Distribution', id='load-monthly-distribution-button', n_clicks=0),
    dcc.Graph(id='monthly-distribution'),
    
    html.P("What is the average value of the numerical column for each day of the month?", style={'fontSize': 25, 'fontWeight': 'bold'}),
    dbc.Row([
        dbc.Col([
            html.Label('Select Numeric Column:', style={'fontSize': 20}),
            dcc.Dropdown(
                id='numeric-column-select19',
                options=[{'label': i, 'value': i} for i in top_columns + [col for col in numeric_columns if col not in top_columns]],
                value=top_columns
            ),
        ], width=6),
        dbc.Col([
    html.Label('Select Year:', style={'fontSize': 20}),
    dcc.Dropdown(
        id='global-year-select10',
        options=[{'label': 'All Years', 'value': 'ALL'}] +
                [{'label': str(year), 'value': year} for year in df['year'].unique()],
        value=df['year'].min()
    ),
    dcc.DatePickerRange(
        id='my-date-picker-range3',
        min_date_allowed=df['date'].min(),
        max_date_allowed=df['date'].max(),
        initial_visible_month=df['date'].min(),
        start_date=df['date'].min(),
        end_date=df['date'].max()
    ),
], width=6)


    ]),
    html.Button('Load/Hide Daily Average Bar', id='load-daily-average-bar-button', n_clicks=0),
    dcc.Graph(id='daily-average-bar'),
    
    html.P("What is the distribution of values in the numerical column for each day of the month?", style={'fontSize': 25, 'fontWeight': 'bold'}),
    dbc.Row([
        dbc.Col([
            html.Label('Select Numeric Column:', style={'fontSize': 20}),
            dcc.Dropdown(
                id='numeric-column-select20',
                options=[{'label': i, 'value': i} for i in top_columns + [col for col in numeric_columns if col not in top_columns]],
                value=top_columns
            ),
        ], width=6),
        dbc.Col([
    html.Label('Select Year:', style={'fontSize': 20}),
    dcc.Dropdown(
        id='global-year-select11',
        options=[{'label': 'All Years', 'value': 'ALL'}] +
                [{'label': str(year), 'value': year} for year in df['year'].unique()],
        value=df['year'].min()
    ),
    dcc.DatePickerRange(
        id='my-date-picker-range4',
        min_date_allowed=df['date'].min(),
        max_date_allowed=df['date'].max(),
        initial_visible_month=df['date'].min(),
        start_date=df['date'].min(),
        end_date=df['date'].max()
    ),
], width=6)


    ]),
    html.Button('Load/Hide Daily Distribution', id='load-daily-distribution-button', n_clicks=0),
    dcc.Graph(id='daily-distribution'),
    
    html.P("How does the value of the numerical column change over time for clients within specific ranges?", style={'fontSize': 25, 'fontWeight': 'bold'}),
    dbc.Row([
        dbc.Col([
            html.Label('Select Numeric Column:', style={'fontSize': 20}),
            dcc.Dropdown(
                id='numeric-column-select21',
                options=[{'label': i, 'value': i} for i in top_columns + [col for col in numeric_columns if col not in top_columns]],
                value=top_columns
            ),
        ], width=6),
        
        dbc.Col([
            html.Label('Select Number of Bins for Value Ranges:', style={'fontSize': 20}),
            dcc.Input(
                id='bin-count-input',
                type='number',
                value=5
            ),
        ], width=6)
    ]),
    html.Button('Load/Hide Range Time Series', id='load-range-time-series-button', n_clicks=0),
    dcc.Graph(id='range-time-series'), 
])

app.clientside_callback(
    """
    function(n_clicks) {
        if(n_clicks % 2 == 1){
            return {'display': 'block'}
        } else {
            return {'display': 'none'}
        }
    }
    """,
    Output('histogram', 'style'),
    Input('load-histogram-button', 'n_clicks')
)

@app.callback(
    Output('histogram', 'figure'),
    [Input('numeric-column-select1', 'value'),
     Input('load-histogram-button', 'n_clicks')]
)
def update_histogram(column, n_clicks):
    if n_clicks > 0 and n_clicks % 2 == 1:
        fig = go.Figure()
        fig.add_trace(go.Histogram(x=df[column], nbinsx=50, name=column))

        # Add lines and annotations for mean, min, and max
        shapes = []
        annotations = []

        # Mean
        shapes.append(dict(type='line', yref='paper', y0=0, y1=1, xref='x', x0=df[column].mean(), x1=df[column].mean()))
        annotations.append(dict(x=df[column].mean(), y=0.9, xref='x', yref='paper', showarrow=False, text='Mean: {:.2f}'.format(df[column].mean()), font=dict(color='black')))

        # Min
        shapes.append(dict(type='line', yref='paper', y0=0, y1=1, xref='x', x0=df[column].min(), x1=df[column].min(), line=dict(color='blue', dash='dash')))
        annotations.append(dict(x=df[column].min(), y=0.8, xref='x', yref='paper', showarrow=False, text='Min: {:.2f}'.format(df[column].min()), font=dict(color='blue')))

        # Max
        shapes.append(dict(type='line', yref='paper', y0=0, y1=1, xref='x', x0=df[column].max(), x1=df[column].max(), line=dict(color='red', dash='dash')))
        annotations.append(dict(x=df[column].max(), y=0.8, xref='x', yref='paper', showarrow=False, text='Max: {:.2f}'.format(df[column].max()), font=dict(color='red')))

        fig.update_layout(title='Histogram, Mean, Min, and Max of {}'.format(column), shapes=shapes, annotations=annotations)

        return fig
    else:
        return {}


    

app.clientside_callback(
    """
    function(n_clicks) {
        if(n_clicks % 2 == 1){
            return {'display': 'block'}
        } else {
            return {'display': 'none'}
        }
    }
    """,
    Output('time-series', 'style'),
    Input('load-time-series-button', 'n_clicks')
)

@app.callback(
    Output('time-series', 'figure'),
    [Input('numeric-column-select2', 'value'),
     Input('agg-func-select', 'value'),
     Input('load-time-series-button', 'n_clicks')]
)
def update_time_series(column, agg_funcs, n_clicks):
    if n_clicks > 0 and n_clicks % 2 == 1:
        df_temp = df.copy()
        df_temp.set_index(Time, inplace=True) 

        fig = go.Figure()

        if not isinstance(agg_funcs, list):
            agg_funcs = [agg_funcs]

        for agg_func in agg_funcs:
            if agg_func in ['std', 'mean', 'median']:
                monthly_data = df_temp[column].resample('M').agg(agg_func)
                fig.add_trace(go.Scatter(x=monthly_data.index, y=monthly_data,
                                        mode='lines', name='Monthly {} of {}'.format(agg_func, column)))
        
        fig.update_layout(title_text='Time Series of {} with Selected Aggregation Functions'.format(column))
        return fig
    else:
        return {}





# Update the group time series graph
app.clientside_callback(
    """
    function(n_clicks) {
        if(n_clicks % 2 == 1){
            return {'display': 'block'}
        } else {
            return {'display': 'none'}
        }
    }
    """,
    Output('group-time-series', 'style'),
    Input('load-group-time-series-button', 'n_clicks')
)

@app.callback(
    Output('group-time-series', 'figure'),
    [Input('numeric-column-select4', 'value'),
     Input('group-column-select1', 'value'),
     Input('load-group-time-series-button', 'n_clicks')]
)
def update_group_time_series(column, group_column, n_clicks=None):
    if n_clicks > 0 and n_clicks % 2 == 1:
        df_temp = df.copy()
        df_temp[Time] = pd.to_datetime(df_temp[Time])
        df_temp.set_index(Time, inplace=True)

        fig = go.Figure()

        # Get unique values of the group column
        groups = df_temp[group_column].unique()

        for group in groups:
            # Calculate aggregation result for each group
            group_data = df_temp[df_temp[group_column] == group][column]
            group_agg = group_data.resample('M').mean()  # Using mean as an example aggregation function, modify as needed
        
            group_agg_sorted = group_agg.sort_index()  # Sort the data by index

            fig.add_trace(go.Scatter(x=group_agg_sorted.index, y=group_agg_sorted, mode='lines', name='{} - {}'.format(group_column, group)))

        fig.update_layout(title_text='Time Series of {} on Different {} Groups'.format(column, group_column))
        return fig
    else:
        return {}


# Update the correlation graph
app.clientside_callback(
    """
    function(n_clicks) {
        if(n_clicks % 2 == 1){
            return {'display': 'block'}
        } else {
            return {'display': 'none'}
        }
    }
    """,
    Output('correlation-graph', 'style'),
    Input('load-correlation-button', 'n_clicks')
)

@app.callback(
    Output('correlation-graph', 'figure'),
    [Input('numeric-column-select5', 'value'),
     Input('numeric-column2-select', 'value'),
     Input('load-correlation-button', 'n_clicks')]
)

def update_correlation_graph(column1, column2, n_clicks=None):
    if n_clicks > 0 and n_clicks % 2 == 1:
        correlation = df[[column1, column2]].corr().iloc[0, 1]  # Calculate correlation coefficient

        fig = go.Figure()
        fig.add_trace(go.Scatter(x=df[column1], y=df[column2], mode='markers', name='Data Points'))  # Plot data points
        fig.update_layout(title='Correlation between {} and {}'.format(column1, column2),
                          xaxis_title=column1, yaxis_title=column2)
        fig.add_annotation(x=0.9, y=0.1, xref='paper', yref='paper', text='Correlation: {:.2f}'.format(correlation),
                           showarrow=False, font=dict(color='black'))  # Add annotation for correlation coefficient

        return fig
    else:
        return {}

app.clientside_callback(
    """
    function(n_clicks) {
        if(n_clicks % 2 == 1){
            return {'display': 'block'}
        } else {
            return {'display': 'none'}
        }
    }
    """,
    Output('weekday-average', 'style'),
    Input('load-weekday-average-button', 'n_clicks')
)

@app.callback(
    Output('weekday-average', 'figure'),
    [Input('numeric-column-select6', 'value'),
     Input('global-year-select1', 'value'),
     Input('date-picker-range-weekday', 'start_date'),
     Input('date-picker-range-weekday', 'end_date'),
     Input('load-weekday-average-button', 'n_clicks')]
)
def update_weekday_average(column, year, start_date, end_date, n_clicks=None):
    if n_clicks > 0 and n_clicks % 2 == 1:
        df_temp = df.copy()

        if year != 'ALL':
            df_temp = df_temp[df_temp['year'] == year]
        
        df_temp = df_temp[(df_temp['date'] >= pd.to_datetime(start_date).date()) & (df_temp['date'] <= pd.to_datetime(end_date).date())]

        weekday_avg = df_temp.groupby('day_of_week')[column].mean()

        fig = go.Figure()
        fig.add_trace(go.Bar(x=weekday_avg.index, y=weekday_avg, name='Average of {}'.format(column)))

        fig.update_layout(title_text='Average of {} for Each Day of the Week in {}'.format(column, year),
                          xaxis_title='Day of the Week', yaxis_title='Average')
        return fig
    else:
        return {}




app.clientside_callback(
    """
    function(n_clicks) {
        if(n_clicks % 2 == 1){
            return {'display': 'block'}
        } else {
            return {'display': 'none'}
        }
    }
    """,
    Output('grouped-box-plot', 'style'),
    Input('load-grouped-box-plot-button', 'n_clicks')
)
@app.callback(
    Output('grouped-box-plot', 'figure'),
    [Input('numeric-column-select8', 'value'),
     Input('group-column-select2', 'value'),
     Input('load-grouped-box-plot-button', 'n_clicks')]
)
def update_grouped_box_plot(column, group_column, n_clicks=None):
    if n_clicks > 0 and n_clicks % 2 == 1:
        fig = go.Figure()

        # Get unique values of the group column in the desired order
        groups = df[group_column].unique()
        groups = sorted(groups)  

        for group in groups:
            group_data = df[df[group_column] == group][column]
            fig.add_trace(go.Box(y=group_data, name='{} - {}'.format(group_column, group)))

        fig.update_layout(title_text='Box Plot of {} for Different {} Groups'.format(column, group_column))
        return fig
    else:
        return {}



app.clientside_callback(
    """
    function(n_clicks) {
        if(n_clicks % 2 == 1){
            return {'display': 'block'}
        } else {
            return {'display': 'none'}
        }
    }
    """,
    Output('day-week-box-plot', 'style'),
    Input('load-day-week-box-plot-button', 'n_clicks')
)
@app.callback(
    Output('day-week-box-plot', 'figure'),
    [
        Input('numeric-column-select10', 'value'),
        Input('global-year-select3', 'value'),
        Input('load-day-week-box-plot-button', 'n_clicks'),
        Input('date-picker-range', 'start_date'),
        Input('date-picker-range', 'end_date')
    ]
)
def update_day_week_box_plot(column, year, n_clicks=None, start_date=None, end_date=None):
    if n_clicks > 0 and n_clicks % 2 == 1:
        if year == 'ALL':
            df_temp = df.copy()
        else:
            df_temp = df[df['year'] == year]

        df_temp = df_temp[(df_temp['date'] >= pd.to_datetime(start_date).date()) & (df_temp['date'] <= pd.to_datetime(end_date).date())]

        fig = go.Figure()

        for day in range(7):  # Assuming Monday=0, Sunday=6
            day_data = df_temp[df_temp['day_of_week'] == day][column]
            fig.add_trace(go.Box(y=day_data, name='Day {}'.format(day + 1)))

        fig.update_layout(title_text='Box Plot of {} for Each Day of the Week in {}'.format(column, year))
        return fig
    else:
        return {}






app.clientside_callback(
    """
    function(n_clicks) {
        if(n_clicks % 2 == 1){
            return {'display': 'block'}
        } else {
            return {'display': 'none'}
        }
    }
    """,
    Output('yearly-average-bar', 'style'),
    Input('load-yearly-average-bar-button', 'n_clicks')
)

@app.callback(
    Output('yearly-average-bar', 'figure'),
    [Input('numeric-column-select11', 'value'),
     Input('load-yearly-average-bar-button', 'n_clicks')]
)
def update_yearly_average_bar(column, n_clicks=None):
    if n_clicks > 0 and n_clicks % 2 == 1:
        df_temp = df.copy()
        df_temp.set_index(Time, inplace=True)
        yearly_avg = df_temp[column].resample('Y').mean()

        fig = go.Figure()

        # Check if yearly average is empty
        if yearly_avg.empty:
            fig.update_layout(title_text='Yearly Average of {} (No data available)'.format(column))
        else:
            fig.add_trace(go.Bar(x=yearly_avg.index.year, y=yearly_avg, name='Yearly Average of {}'.format(column)))
            fig.update_layout(title_text='Yearly Average of {}'.format(column))

        return fig
    else:
        return {}


app.clientside_callback(
    """
    function(n_clicks) {
        if(n_clicks % 2 == 1){
            return {'display': 'block'}
        } else {
            return {'display': 'none'}
        }
    }
    """,
    Output('quarterly-box-plot', 'style'),
    Input('load-quarterly-box-plot-button', 'n_clicks')
)

@app.callback(
    Output('quarterly-box-plot', 'figure'),
    [Input('numeric-column-select12', 'value'),
     Input('global-year-select4', 'value'),
     Input('load-quarterly-box-plot-button', 'n_clicks')]
)
def update_quarterly_box_plot(column, year, n_clicks=None):
    if n_clicks > 0 and n_clicks % 2 == 1:
        if year == 'ALL':
            df_year = df.copy()
        else:
            df_year = df[df['year'] == year]

        fig = go.Figure()

        for quarter in range(1, 5):  # 1 to 4 quarters
            quarter_data = df_year[df_year['quarter'] == quarter][column]
            fig.add_trace(go.Box(y=quarter_data, name='Quarter {}'.format(quarter)))

        fig.update_layout(title_text='Box Plot of {} for Each Quarter of the Year in {}'.format(column, year))
        return fig
    else:
        return {}


app.clientside_callback(
    """
    function(n_clicks) {
        if(n_clicks % 2 == 1){
            return {'display': 'block'}
        } else {
            return {'display': 'none'}
        }
    }
    """,
    Output('day-of-week-trend', 'style'),
    Input('load-day-of-week-trend-button', 'n_clicks')
)
@app.callback(
    Output('day-of-week-trend', 'figure'),
    [Input('numeric-column-select13', 'value'),
     Input('global-year-select5', 'value'),
     Input('load-day-of-week-trend-button', 'n_clicks'),
     Input('my-date-picker-range1', 'start_date'),
     Input('my-date-picker-range1', 'end_date')]
)
def update_day_of_week_trend(column, year, n_clicks=None, start_date=None, end_date=None):
    if n_clicks > 0 and n_clicks % 2 == 1:
        if year == 'ALL':
            df_year = df.copy()
        else:
            start_date = pd.to_datetime(start_date).date() if start_date else None
            end_date = pd.to_datetime(end_date).date() if end_date else None
            df_year = df[(df['year'] == year) & 
                         (df['date'] >= start_date if start_date else df['date']) &
                         (df['date'] <= end_date if end_date else df['date'])]
        
        fig = go.Figure()
        df_grouped = df_year.groupby('day_of_week')[column].mean()
        fig.add_trace(go.Scatter(x=df_grouped.index, y=df_grouped.values, mode='lines'))

        fig.update_layout(title='Trend in {} Over Days of the Week in {}'.format(column, year))
        return fig
    else:
        return {}




app.clientside_callback(
    """
    function(n_clicks) {
        if(n_clicks % 2 == 1){
            return {'display': 'block'}
        } else {
            return {'display': 'none'}
        }
    }
    """,
    Output('yearly-distribution', 'style'),
    Input('load-yearly-distribution-button', 'n_clicks')
)

@app.callback(
    Output('yearly-distribution', 'figure'),
    [Input('numeric-column-select14', 'value'),
     Input('load-yearly-distribution-button', 'n_clicks')]
)
def update_yearly_distribution(column, n_clicks=None):
    if n_clicks > 0 and n_clicks % 2 == 1:
        fig = go.Figure()
        for year in df['year'].unique():
            fig.add_trace(go.Box(y=df[df['year'] == year][column], name=str(year)))
        fig.update_layout(title='Yearly Distribution of {}'.format(column))
        return fig
    else:
        return {}


app.clientside_callback(
    """
    function(n_clicks) {
        if(n_clicks === undefined || n_clicks === null || n_clicks % 2 == 0){
            return {'display': 'none'}
        } else {
            return {'display': 'block'}
        }
    }
    """,
    Output('monthly-average-bar', 'style'),
    Input('load-monthly-average-bar-button', 'n_clicks')
)
@app.callback(
    Output('monthly-average-bar', 'figure'),
    [Input('numeric-column-select15', 'value'),
     Input('global-year-select6', 'value'),
     Input('load-monthly-average-bar-button', 'n_clicks')]
)
def update_monthly_avg_bar(column, year, n_clicks=None):
    if n_clicks > 0 and n_clicks % 2 == 1:
        if year == 'ALL':
            df_temp = df.copy()
        else:
            df_temp = df[df['year'] == year].copy()
        df_grouped = df_temp.groupby('month')[column].mean()
        fig = go.Figure()
        fig.add_trace(go.Bar(x=df_grouped.index, y=df_grouped.values, name='Average of {}'.format(column)))

        fig.update_layout(title_text='Average {} for Each Month in {}'.format(column, year),
                          xaxis_title='Month', yaxis_title='Average')
        return fig
    else:
        return {}



app.clientside_callback(
    """
    function(n_clicks) {
        if(n_clicks % 2 == 1){
            return {'display': 'block'}
        } else {
            return {'display': 'none'}
        }
    }
    """,
    Output('weekly-distribution', 'style'),
    Input('load-weekly-distribution-button', 'n_clicks')
)
@app.callback(
    Output('weekly-distribution', 'figure'),
    [Input('numeric-column-select16', 'value'),
     Input('global-year-select7', 'value'),
     Input('load-weekly-distribution-button', 'n_clicks')]
)
def update_weekly_distribution(column, year, n_clicks=None):
    if n_clicks > 0 and n_clicks % 2 == 1:
        if year == 'ALL':
            df_temp = df.copy()
        else:
            df_temp = df[df['year'] == year]
        
        fig = go.Figure()
        for week in sorted(df_temp['week'].unique()):
            fig.add_trace(go.Box(y=df_temp[df_temp['week'] == week][column], name=str(week)))
        fig.update_layout(title='Weekly Distribution of {} in {}'.format(column, year))
        return fig
    else:
        return {}



app.clientside_callback(
    """
    function(n_clicks) {
        if(n_clicks % 2 == 1){
            return {'display': 'block'}
        } else {
            return {'display': 'none'}
        }
    }
    """,
    Output('day-of-month-trend', 'style'),
    Input('load-day-of-month-trend-button', 'n_clicks')
)
@app.callback(
    Output('day-of-month-trend', 'figure'),
    [Input('numeric-column-select17', 'value'),
     Input('global-year-select8', 'value'),
     Input('load-day-of-month-trend-button', 'n_clicks'),
     Input('my-date-picker-range2', 'start_date'),
     Input('my-date-picker-range2', 'end_date')]
)
def update_day_of_month_trend(column, year, n_clicks=None, start_date=None, end_date=None):
    if n_clicks > 0 and n_clicks % 2 == 1:
        if year == 'ALL':
            df_temp = df.copy()
        else:
            start_date = pd.to_datetime(start_date).date() if start_date else None
            end_date = pd.to_datetime(end_date).date() if end_date else None
            df_temp = df[(df['year'] == year) & 
                         (df['date'] >= start_date if start_date else df['date']) &
                         (df['date'] <= end_date if end_date else df['date'])]
        
        fig = go.Figure()
        df_grouped = df_temp.groupby('day')[column].mean()
        fig.add_trace(go.Scatter(x=df_grouped.index, y=df_grouped.values, mode='lines'))

        fig.update_layout(title='Trend in {} Over Days of the Month in {}'.format(column, year))
        return fig
    else:
        return {}



app.clientside_callback(
    """
    function(n_clicks) {
        if(n_clicks % 2 == 1){
            return {'display': 'block'}
        } else {
            return {'display': 'none'}
        }
    }
    """,
    Output('monthly-distribution', 'style'),
    Input('load-monthly-distribution-button', 'n_clicks')
)
@app.callback(
    Output('monthly-distribution', 'figure'),
    [Input('numeric-column-select18', 'value'),
     Input('global-year-select9', 'value'),
     Input('load-monthly-distribution-button', 'n_clicks')]
)
def update_monthly_distribution(column, year, n_clicks=None):
    if n_clicks > 0 and n_clicks % 2 == 1:
        if year == 'ALL':
            df_temp = df.copy()
        else:
            df_temp = df[df['year'] == year]

        fig = go.Figure()
        for month in np.sort(df_temp['month'].unique()):
            fig.add_trace(go.Violin(y=df_temp[df_temp['month'] == month][column], name=str(month), box_visible=True))
        fig.update_layout(title='Monthly Distribution of {} in {}'.format(column, year))
        return fig
    else:
        return {}



app.clientside_callback(
    """
    function(n_clicks) {
        if(n_clicks % 2 == 1){
            return {'display': 'block'}
        } else {
            return {'display': 'none'}
        }
    }
    """,
    Output('daily-average-bar', 'style'),
    Input('load-daily-average-bar-button', 'n_clicks')
)

@app.callback(
    Output('daily-average-bar', 'figure'),
    [Input('numeric-column-select19', 'value'),
     Input('global-year-select10', 'value'),
     Input('load-daily-average-bar-button', 'n_clicks'),
     Input('my-date-picker-range3', 'start_date'),
     Input('my-date-picker-range3', 'end_date')]
)
def update_daily_avg_bar(column, year, n_clicks=None, start_date=None, end_date=None):
    if n_clicks > 0 and n_clicks % 2 == 1:
        if year == 'ALL':
            df_temp = df.copy()
        else:
            start_date = pd.to_datetime(start_date).date() if start_date else None
            end_date = pd.to_datetime(end_date).date() if end_date else None
            df_temp = df[(df['year'] == year) & 
                         (df['date'] >= start_date if start_date else df['date']) &
                         (df['date'] <= end_date if end_date else df['date'])]
        
        fig = go.Figure()
        df_grouped = df_temp.groupby('day')[column].mean().sort_index()
        fig.add_trace(go.Bar(x=df_grouped.index, y=df_grouped.values))

        fig.update_layout(title='Average {} for Each Day of the Month in {}'.format(column, year))
        return fig
    else:
        return {}




app.clientside_callback(
    """
    function(n_clicks) {
        if(n_clicks % 2 == 1){
            return {'display': 'block'}
        } else {
            return {'display': 'none'}
        }
    }
    """,
    Output('daily-distribution', 'style'),
    Input('load-daily-distribution-button', 'n_clicks')
)
@app.callback(
    Output('daily-distribution', 'figure'),
    [Input('numeric-column-select20', 'value'),
     Input('global-year-select11', 'value'),
     Input('load-daily-distribution-button', 'n_clicks'),
     Input('my-date-picker-range4', 'start_date'),
     Input('my-date-picker-range4', 'end_date')]
)
def update_daily_distribution(column, year, n_clicks=None, start_date=None, end_date=None):
    if n_clicks > 0 and n_clicks % 2 == 1:
        if year == 'ALL':
            df_temp = df.copy()
        else:
            start_date = pd.to_datetime(start_date).date() if start_date else None
            end_date = pd.to_datetime(end_date).date() if end_date else None
            df_temp = df[(df['year'] == year) & 
                         (df['date'] >= start_date if start_date else df['date']) &
                         (df['date'] <= end_date if end_date else df['date'])]
        
        fig = go.Figure()
        for day in np.sort(df_temp['day'].unique()):
            fig.add_trace(go.Violin(y=df_temp[df_temp['day'] == day][column], name=str(day), box_visible=True))
        fig.update_layout(title='Daily Distribution of {} in {}'.format(column, year))
        return fig
    else:
        return {}


app.clientside_callback(
    """
    function(n_clicks) {
        if(n_clicks === undefined || n_clicks === null || n_clicks % 2 == 0){
            return {'display': 'none'}
        } else {
            return {'display': 'block'}
        }
    }
    """,
    Output('range-time-series', 'style'),
    Input('load-range-time-series-button', 'n_clicks')
)
@app.callback(
    Output('range-time-series', 'figure'),
    [Input('numeric-column-select21', 'value'),
     Input('bin-count-input', 'value'),
     Input('load-range-time-series-button', 'n_clicks')]
)
def update_range_time_series(column, bin_count, n_clicks=None):
    if n_clicks > 0 and n_clicks % 2 == 1:
        df_temp = df.copy()
        df_temp[Time] = pd.to_datetime(df_temp[Time])
        df_temp.set_index(Time, inplace=True)

        # Bin the numeric column
        df_temp['{}_bin'.format(column)] = pd.qcut(df_temp[column], q=bin_count)

        # Sort the bins by their left boundary
        bins_boundaries = df_temp['{}_bin'.format(column)].unique()
        bins_boundaries_sorted = sorted(bins_boundaries, key=lambda x: x.left)

        # Convert bin to categorical and sort
        df_temp['{}_bin'.format(column)] = pd.Categorical(df_temp['{}_bin'.format(column)], ordered=True, categories=bins_boundaries_sorted)

        fig = go.Figure()

        for bin in bins_boundaries_sorted:  # Iterate over sorted bins
            # Calculate aggregation result for each bin
            bin_data = df_temp[df_temp['{}_bin'.format(column)] == bin][column]
            bin_agg = bin_data.resample('M').mean()  # Using mean as an example aggregation function, modify as needed
            
            bin_agg_sorted = bin_agg.sort_index()  # Sort the data by index

            fig.add_trace(go.Scatter(x=bin_agg_sorted.index, y=bin_agg_sorted, mode='lines', name='{} - {}'.format(column, bin)))

        fig.update_layout(title_text='Time Series of {} for Different Value Ranges'.format(column))
        return fig
    else:
        return {}

app.run_server(mode='external')



NameError: name 'df' is not defined